In [ ]:
import glob
import numpy as np
a = np.zeros((5, 5))
session=0
for name in glob.glob('/Volumes/GRATTONLAB/MSC/MSC_Dconns/Memory/*.dconn.nii'):
    print(name)
    np.savetxt(name+str(session)+'test.csv',a,delimiter=',')
    session=session+1

In [ ]:
import warnings
import sys 
if not sys.warnoptions:
    warnings.simplefilter("ignore")
    
# Import libraries
import nibabel as nib
import numpy as np
import os 
import time
from nilearn import plotting
from sklearn.model_selection import StratifiedKFold, cross_val_score
#import nilearn.decoding
from sklearn.linear_model import RidgeClassifier
from sklearn.model_selection import KFold
cv = KFold(n_splits=4)
clf= RidgeClassifier()



data_file='/Users/Alexis/Desktop/MSC_4k_surfaces/MSC01.L.inflated.4k.surf.gii'
data_file = nib.load(data_file)
task_data = data_file.get_data()

In [ ]:
data_file='/Volumes/GRATTONLAB/MSC/MSC_Dconns/Memory/MSC01_mem_AllSessions_cortex.dconn.nii'
data_file = nib.load(data_file)
task_data = data_file.get_data()
"""
rest_file='/Volumes/GRATTONLAB/MSC/MSC_Dconns/Rest/MSC01_REST_AllSessions_cortex.dconn.nii'
rest_file = nib.load(rest_file)
rest_data = rest_file.get_data()

mask_file='/Users/Alexis/Desktop/BarchSZ/bottomBrainMask.dtseries.nii'
mask_file=nib.load(mask_file)
mask=mask_file.get_data()
"""


In [ ]:
# Surface based searchlight
# ----------------------
from nilearn import datasets, surface
from sklearn import neighbors
from nilearn.decoding.searchlight import search_light

#right/left gift template
coords,_=surface.load_surf_mesh('/Users/Alexis/Desktop/MSC_4k_surfaces/Conte69.L.inflated.4k_fs_LR.surf.gii')
Right_mesh=surface.load_surf_mesh('/Users/Alexis/Desktop/MSC_4k_surfaces/Conte69.R.inflated.4k_fs_LR.surf.gii')



radius = 3.
nn = neighbors.NearestNeighbors(radius=radius)
adjacency = nn.fit(coords).radius_neighbors_graph(coords).tolil()

data_file='/Users/Alexis/Desktop/MSC01_ses1.dconn.nii'
data_file = nib.load(data_file)
task_data = data_file.get_data()

fake_rest=data_file.get_data()

taskSize=task_data.shape[0]
#restSize=rest_data.shape[0]

t=np.ones(taskSize, dtype=int)
#r=np.zeros(restSize, dtype=int)

#x=np.dstack((task_data, rest_data))
#y=np.concatenate((t,r))




scores = search_light(task_data, t, clf, adjacency, cv=cv, n_jobs=1)

In [ ]:
# Fetch a coarse surface of the left hemisphere only for speed
fsaverage = datasets.fetch_surf_fsaverage(mesh='fsaverage5')
hemi = 'left'

# To define the BOLD responses to be included within each searchlight "sphere"
# we define an adjacency matrix based on the inflated surface vertices such
# that nearby surfaces are concatenated within the same searchlight.

infl_mesh = fsaverage['infl_' + hemi]
coords, _ = surface.load_surf_mesh(infl_mesh)
radius = 3.
nn = neighbors.NearestNeighbors(radius=radius)
adjacency = nn.fit(coords).radius_neighbors_graph(coords).tolil()

taskSize=task_data.shape[0]
restSize=rest_data.shape[0]

t=np.ones(taskSize, dtype=int)
r=np.zeros(restSize, dtype=int)

#x=np.dstack((task_data, rest_data))
y=np.concatenate((t,r))




scores = search_light(rest_data, r, clf, adjacency, cv=cv, n_jobs=1)

In [ ]:
def volume_from_cifti(data, axis):
    assert isinstance(axis, nib.cifti2.BrainModelAxis)
    data = data.T[axis.volume_mask]                          # Assume brainmodels axis is last, move it to front
    volmask = axis.volume_mask                               # Which indices on this axis are for voxels?
    vox_indices = tuple(axis.voxel[axis.volume_mask].T)      # ([x0, x1, ...], [y0, ...], [z0, ...])
    vol_data = np.zeros(axis.volume_shape + data.shape[1:],  # Volume + any extra dimensions
                        dtype=data.dtype)
    vol_data[vox_indices] = data                             # "Fancy indexing"
    return nib.Nifti1Image(vol_data, axis.affine)             # Add affine for spatial interpretation

In [ ]:
volume_from_cifti(cifti_data, axes[1]).orthoview()

In [ ]:
def surf_data_from_cifti(data, axis, surf_name):
    assert isinstance(axis, nb.cifti2.BrainModelAxis)
    for name, data_indices, model in axis.iter_structures():  # Iterates over volumetric and surface structures
        if name == surf_name:                                 # Just looking for a surface
            data = data.T[data_indices]                       # Assume brainmodels axis is last, move it to front
            vtx_indices = model.vertex                        # Generally 1-N, except medial wall vertices
            surf_data = np.zeros((vtx_indices.max() + 1,) + data.shape[1:], dtype=data.dtype)
            surf_data[vtx_indices] = data
            return surf_data
    raise ValueError(f"No structure named {surf_name}")

In [ ]:
_ = nlp.plot_surf(str(data_dir / "conte69/Conte69.L.inflated.32k_fs_LR.surf.gii"),
                  surf_data_from_cifti(cifti_data, axes[1], 'CIFTI_STRUCTURE_CORTEX_LEFT').mean(axis=1),
                  cmap='plasma')

In [ ]:
def decompose_cifti(img):
    data = img.get_fdata(dtype=np.float32)
    brain_models = img.header.get_axis(1)  # Assume we know this
    return (volume_from_cifti(data, brain_models),
            surf_data_from_cifti(data, brain_models, "CIFTI_STRUCTURE_CORTEX_LEFT"),
            surf_data_from_cifti(data, brain_models, "CIFTI_STRUCTURE_CORTEX_RIGHT"))

In [ ]:
vol, left, right = decompose_cifti(cifti)
print(vol.shape, left.shape, right.shape)

In [ ]:
scores
#rest_file.header.get_axis(0)

In [ ]:
searchlight = nilearn.decoding.SearchLight(
    mask,
    radius=5.6, estimator=clf,
    n_jobs=n_jobs,
    verbose=1, cv=cv)
searchlight.fit(data_file, t)

In [ ]:
import pandas as pd
from nilearn import datasets
from nilearn.image import new_img_like, load_img, get_data

# We fetch 2nd subject from haxby datasets (which is default)
haxby_dataset = datasets.fetch_haxby()

# print basic information on the dataset
print('Anatomical nifti image (3D) is located at: %s' % haxby_dataset.mask)
print('Functional nifti image (4D) is located at: %s' % haxby_dataset.func[0])

fmri_filename = haxby_dataset.func[0]
labels = pd.read_csv(haxby_dataset.session_target[0], sep=" ")
y = labels['labels']
session = labels['chunks']

In [ ]:
from nilearn.image import index_img
condition_mask = y.isin(['face', 'house'])

fmri_img = index_img(fmri_filename, condition_mask)
y, session = y[condition_mask], session[condition_mask]

In [ ]:
import numpy as np

mask_img = load_img(haxby_dataset.mask)

# .astype() makes a copy.
process_mask = get_data(mask_img).astype(np.int)
picked_slice = 29
process_mask[..., (picked_slice + 1):] = 0
process_mask[..., :picked_slice] = 0
process_mask[:, 30:] = 0
process_mask_img = new_img_like(mask_img, process_mask)

In [ ]:
test=fmri_img.header


In [ ]:
# Make processing parallel
# /!\ As each thread will print its progress, n_jobs > 1 could mess up the
#     information output.
n_jobs = 1
from sklearn.linear_model import RidgeClassifier
from sklearn.model_selection import KFold
cv = KFold(n_splits=4)
clf= RidgeClassifier()
import nilearn.decoding
# The radius is the one of the Searchlight sphere that will scan the volume
searchlight = nilearn.decoding.SearchLight(
    mask_img,
    radius=5.6, estimator=clf,
    n_jobs=n_jobs,
    verbose=1, cv=cv)
searchlight.fit(fmri_img, y)

In [ ]:
from nilearn import image
mean_fmri = image.mean_img(fmri_img)

from nilearn.plotting import plot_stat_map, plot_img, show
searchlight_img = new_img_like(mean_fmri, searchlight.scores_)

# Because scores are not a zero-center test statistics, we cannot use
# plot_stat_map
plot_img(searchlight_img, bg_img=mean_fmri,
         title="Searchlight", display_mode="z", cut_coords=[-9],
         vmin=.42, cmap='hot', threshold=.2, black_bg=True)

In [ ]:
from nilearn import surface

#test=surface.load_surf_data('/Volumes/GRATTONLAB/MSC/MSC_Dconns/Rest/MSC01_REST_AllSessions_cortex.dconn.nii')

In [ ]:

#import relevant packages
#this script will just start off with doing a simple CV calculation
from sklearn.linear_model import RidgeClassifier
import numpy as np
import os
import sys
import pandas as pd
import warnings
warnings.filterwarnings("ignore")
import itertools
from sklearn.model_selection import cross_val_score
import nibabel as nib
import glob
import time
#using for now; because dconns are all the same size
vertex_size=59412
scores=np.empty((1,vertex_size))
clf=RidgeClassifier()
#iterate through each index
#lets try a different approach
#first load all data
#will have to determine if there is a way to declare size based on files found in glob function
allTask=np.empty((vertex_size,vertex_size,10))
allRest=np.empty((vertex_size,vertex_size,10))


allTask.shape

import numpy as np
test=np.genfromtxt('~/Desktop/MSC01_sess9.dconn.nii1first.csv',delimiter=',')


test=np.genfromtxt('/Users/Alexis/Desktop/CV_mem_MSC01.csv',delimiter=',')

In [9]:
import reshape
import pandas as pd
import numpy as np
import os
import sys
thisDir = os.path.expanduser('~/Desktop/MSC_Alexis/analysis/')
dataDir = thisDir + 'data/mvpa_data/'
restFC=reshape.matFiles(dataDir+'rest/corrmats_timesplit/fourths/MSC01_parcel_corrmat.mat') 

In [11]:
restFC=np.reshape(restFC,(10,4,55278))

In [12]:
test=np.reshape(restFC,(-1,55278))

In [13]:
test.shape

(40, 55278)

In [2]:
import permuteROI
import numpy as np
res=permuteROI.modelAll_byRow('MSC03')

res.tofile('/Users/Alexis/Desktop/MSC_Alexis/analysis/output/results/permutation/ALL/MSC03_Row.csv', sep = ',')